# Verge and foliot mechanism

References:

- https://github.com/JuliaReach/ReachabilityModels.jl/issues/52

- https://github.com/SciML/DifferentialEquations.jl/issues/703

## Model parameters

In [2]:
using ReachabilityAnalysis, LinearAlgebra, Plots, IntervalConstraintProgramming
const IA = IntervalArithmetic
LazySets.set_ztol(Float64, 1e-12)

1.0e-12

In [3]:
#plotly()

In [4]:
const torque = 1.0
const Ic = 10.0
const Iv = 0.15
const r_c = 1.0
const r_v = 0.3
const αc = 24.0 * pi/180
const e_r = 0.05
const nteeth = round(Int, 2pi/αc) # n
const Mv = Iv/r_v^2
const Mc = Ic/r_c^2
const Gc = Mv*(1 + e_r)/r_c/(Mv + Mc)
const Gv = Mc*(1 + e_r)/r_v/(Mv + Mc)
const αv = Gv*r_c/(1-e_r)*αc/2

p = (torque, Ic);

## Guards

In [5]:
#plotly()

In [6]:
upper_trig(m) = @constraint $r_c*sin(x1 - ($m-1)* ($αc)) - $r_v*tan(x2 + $αv/2) == 0

lower_trig(m) = @constraint $r_c*sin(($m-1)* ($αc) - x1) - $r_v*tan(-x2 + $αv/2) == 0

#dom = IntervalBox(-0.2 .. 0.2, 2)
dom = (-0.2 .. 0.2) × (-2 .. 2)

ε = 0.05
U = [pave(upper_trig(i), dom, ε) for i in 1:nteeth];
L = [pave(lower_trig(i), dom, ε) for i in 1:nteeth];

In [12]:
fig = plot(xlab="x₁", ylab="x₂", title="Upper trig")
[plot!(fig, U[i].boundary, lab="") for i in eachindex(U)]
fig

<!DOCTYPE html>
 
 
 Plots.jl

In [13]:
plot(rand(6))

<!DOCTYPE html>
 
 
 Plots.jl

In [18]:
fig = plot()
plot!(fig, rand(Zonotope))

<!DOCTYPE html>
 
 
 Plots.jl

In [17]:
Plots.inline()

<!DOCTYPE html>
 
 
 Plots.jl

## Transition map

In [ ]:
# σ = 1
Fupper = zeros(5, 5)
Fupper[3, 3] = -r_c*Gc
Fupper[3, 4] = r_v*Gc
Fupper[4, 3] = r_c*Gv
Fupper[4, 4] = - r_v*Gv

# σ = -1
Flower = zeros(5, 5)
Flower[3, 3] = -r_c*Gc
Flower[3, 4] = -r_v*Gc
Flower[4, 3] = -r_c*Gv
Flower[4, 4] = - r_v*Gv;

## Continuous post-operator

In [ ]:
A = zeros(5, 5) # variables x1, x2, x3, x4 and the last variable wraps the constant input term
A[1, 3] = 1.0
A[2, 4] = 1.0
A[3, 5] = 1.0

X0 = Singleton([0, 0, 0, 3.0, p[1]/p[2]])
prob = @ivp(x' = A*x, x(0) ∈ X0);

sol = solve(prob, tspan=(0.0, 0.2))

plot(sol, vars=(0, 1), xlab="t", ylab="x₁(t)")

In [ ]:
# overapproximate a nonlinear constraint with an HPolyhedron
dom = IntervalBox(IA.Interval(-10, 10), IA.Interval(-10, 10))
m = 1
C = @constraint $r_c*sin(x - ($m-1)* ($αc)) - $r_v*tan(y + $αv/2) == 0

pav = pave(C, dom, 0.1)

plot(pav.boundary, lab="boundary", legend=:bottomright)

typeof(pav.boundary)

Y = ConvexHullArray([convert(Hyperrectangle, X) for X in pav.boundary]);

plot(pav.boundary, lab="boundary", legend=:bottomright)
plot!(Y, lab="convex hull")

dirs = OctDirections(2)
H = overapproximate(pav, dirs);

plot(pav.boundary, ratio=1)
plot!(H, alpha=.4, lab="oct directions")

In [ ]:
plot!(fig, sol, vars=(1, 2), xlab="x₁", ylab="x₂")

For each value of $m$ and a choice between uppper / lower, there is (1) the trigonometric constraint, (2) the half-space that involves variables $x_3$ and $x_4$, and (3) constraints on the domain of $x_1$.

We can combine (3) into the domain of (1).

In [ ]:
upper_dom_x1(m; p=0) = ((m-1-1/2)*αc .. (m-1+1/2)*αc) + (2p)*pi # in principle there are more domains multiples of 2pi
lower_dom_x1(m; p=0) = ((m-1-1/2)*αc .. (m-1+1/2)*αc) + (2p-1)*pi # in principle there are more domains multiples of 2pi

# the admissible domain of x2 is given 
upper_dom(m; p=0) = upper_dom_x1(m, p=p) × (-2 .. 2)
lower_dom(m; p=0) = lower_dom_x1(m, p=p) × (-2 .. 2)

In [ ]:
upper_dom_x1(1, p=0)

In [ ]:
@show nteeth
U = [pave(upper_trig(m), upper_dom(m, p=0), 0.05) for m in 1:nteeth];
L = [pave(lower_trig(m), lower_dom(m, p=0), 0.05) for m in 1:nteeth];

fig = plot(xlab="x₁", ylab="x₂", title="Upper trig constraint (p = 0)")
[plot!(fig, U[i].boundary, lab="") for i in eachindex(U)]
fig

In [ ]:
U = pave(upper_trig(1), upper_dom(1, p=0), 0.05)
U.boundary

In [ ]:
upper_dom(1, p=0)

In [ ]:
plot(U.boundary)

In [ ]:
U = [pave(upper_trig(m), upper_dom(m, p=1), 0.1) for m in 0:nteeth];
L = [pave(lower_trig(m), lower_dom(m, p=1), 0.1) for m in 0:nteeth];

fig = plot(xlab="x₁", ylab="x₂", title="Upper trig constraint (p = 1)")
[plot!(fig, U[i].boundary, lab="") for i in eachindex(U)]
fig

In [ ]:
pave(upper_trig(m), upper_dom(1, p=1), 0.1)

The integer $p$ just translates the constraints to the right, so we can consider $p = 0$ for the flowpipe-guard intersection until the last guard for $p = 0$ is enabled.

In [ ]:
U = [pave(upper_trig(m), upper_dom(m, p=0), 0.1) for m in 0:nteeth];
L = [pave(lower_trig(m), lower_dom(m, p=0), 0.1) for m in 0:nteeth];

fig = plot(xlab="x₁", ylab="x₂", title="Lower trig constraint (p = 0)")
[plot!(fig, L[i].boundary, lab="") for i in eachindex(U)]
fig

In [ ]:
@show nteeth
U = [pave(upper_trig(m), upper_dom(m, p=0), 0.1) for m in 0:nteeth];
L = [pave(lower_trig(m), lower_dom(m, p=0), 0.1) for m in 0:nteeth];

fig = plot(xlab="x₁", ylab="x₂", title="Trig constraints")
plot!(fig, U[1].boundary, lab="Upper constraints", color=:blue)
[plot!(fig, U[i].boundary, lab="", color=:blue) for i in 2:length(U)]

plot!(fig, L[1].boundary, lab="Lower constraints", color=:red)
[plot!(fig, L[i].boundary, lab="", color=:red) for i in 2:length(U)]

fig

In [ ]:
plot!(sol, vars=(1, 2), xlab="x₁", ylab="x₂")